# NRPy+ Tutorial
## Part 1: Generating C code to solve the scalar wave equation, in Cartesian coordinates

We wish to solve the scalar wave equation in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as-yet unspecified) and $c$ is the wave speed,

subject to some initial condition
$$u(0,x,y,...) = f(x,y,...)$$

and suitable, approximate spatial boundary conditions.

Define 
$$v(t,x,y,...) = \partial_t u(t,x,y,...)$$

Then the second-order PDE is reduced to a set of two coupled first-order PDEs

$$\partial_t u = v$$
$$\partial_t v = c^2 \nabla^2 u \text{,}$$

which we can step forward in time using the Method of Lines. As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy+ via finite differencing.

In [ ]:
import NRPy_param_funcs as par
import tensor as ten
import grid as gri
import finite_difference as fin

In [2]:
DIM = par.parval_from_str("DIM")

uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])
uu_dDD = ten.declarerank2("uu_dDD","sym12")
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

fin.FD_outputC_ToFile(str(thismodule)+"/NRPy_codegen/scalarwave_RHS.h",
                      [lhrh(lhs=gri.gfaccess("out_gfs","UU_rhs"),rhs=uu_rhs),
                       lhrh(lhs=gri.gfaccess("out_gfs","VV_rhs"),rhs=vv_rhs)])

/*
 *  Original SymPy expressions:
 *  "[const REAL_SIMD_ARRAY uu_dDD11 = invdx1**2*(-5*uu/2 + 4*uu_i0_i1m1_i2/3 - uu_i0_i1m2_i2/12 + 4*uu_i0_i1p1_i2/3 - uu_i0_i1p2_i2/12),
 *    const REAL_SIMD_ARRAY uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1_i1_i2/3 - uu_i0m2_i1_i2/12 + 4*uu_i0p1_i1_i2/3 - uu_i0p2_i1_i2/12),
 *    const REAL_SIMD_ARRAY uu_dDD22 = invdx2**2*(-5*uu/2 + 4*uu_i0_i1_i2m1/3 - uu_i0_i1_i2m2/12 + 4*uu_i0_i1_i2p1/3 - uu_i0_i1_i2p2/12),
 *    const REAL_SIMD_ARRAY __RHS_exp_0 = vv,
 *    const REAL_SIMD_ARRAY __RHS_exp_1 = uu_dDD00*wavespeed**2 + uu_dDD11*wavespeed**2 + uu_dDD22*wavespeed**2]"
 */
{
   const REAL_SIMD_ARRAY uu_i0_i1_i2m2 = ReadSIMD(&UUGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2-2)]);
   const REAL_SIMD_ARRAY uu_i0_i1_i2m1 = ReadSIMD(&UUGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2-1)]);
   const REAL_SIMD_ARRAY uu_i0_i1m2_i2 = ReadSIMD(&UUGF[CCTK_GFINDEX3D(cctkGH, i0,i1-2,i2)]);
   const REAL_SIMD_ARRAY uu_i0_i1m1_i2 = ReadSIMD(&UUGF[CCTK_GFINDEX3D(cctkGH, i0,i1-1,i2)]);
   const 